In [3]:
import pandas as pd

In [4]:
df = pd.read_csv("asl_landmarks.csv")
print("Total samples: ", len(df))
df.head()

Total samples:  2497


,label,x0,y0,z0,x1,y1,z1,x2,y2,z2,...,z17,x18,y18,z18,x19,y19,z19,x20,y20,z20
0,A,0.593800,0.798356,-7.649077e-07,0.503934,0.740276,-0.019193,0.444183,0.631012,-0.027275,...,-0.028513,0.618038,0.509346,-0.051375,0.605658,0.579815,-0.035927,0.605530,0.627095,-0.015392
1,A,0.730166,0.617582,-5.512676e-07,0.639415,0.565125,-0.029493,0.577366,0.462825,-0.037581,...,-0.006989,0.769189,0.340501,-0.034803,0.759857,0.405193,-0.023299,0.747320,0.447488,-0.003749
2,A,0.427747,0.586118,-8.728184e-07,0.332063,0.539541,-0.013546,0.258781,0.429579,-0.019302,...,-0.032051,0.426942,0.266215,-0.055565,0.421107,0.343477,-0.038379,0.433551,0.399313,-0.016079
3,A,0.468265,0.601081,-8.472105e-07,0.369813,0.546696,-0.006734,0.302968,0.442877,-0.009222,...,-0.030330,0.476178,0.295535,-0.046805,0.467783,0.366750,-0.029050,0.475853,0.420130,-0.009718
4,A,0.528097,0.625915,-7.491166e-07,0.433816,0.579672,-0.020089,0.362457,0.470722,-0.027540,...,-0.027452,0.545174,0.317821,-0.050836,0.536625,0.386965,-0.036382,0.537917,0.441104,-0.017064


In [5]:
X = df.drop(["label"], axis = 1).values
y = df["label"].values

print("X shape: ", X.shape)
print("y shape: ", y.shape)

X shape:  (2497, 63)
y shape:  (2497,)


In [6]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

le = LabelEncoder()
y_encoded = le.fit_transform(y)

print("Label Mapping: ", dict(zip(le.classes_, le.transform(le.classes_))))

Label Mapping:  {'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4}


In [7]:
X_train, X_test, y_train, y_test = train_test_split(
  X, y_encoded, test_size = 0.2, random_state = 42, shuffle=True
)

print("Training samples: ", X_train.shape)
print("Testing samples: ", X_test.shape)

Training samples:  (1997, 63)
Testing samples:  (500, 63)


In [8]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam

model = Sequential([
  Dense(256, activation='relu', input_shape=(63,)),
  Dropout(0.3),

  Dense(128, activation='relu'),
  Dropout(0.3),

  Dense(64, activation='relu'),

  Dense(5, activation='softmax')
])

In [9]:
model.compile(
  optimizer = Adam(learning_rate=0.001),
  loss = 'sparse_categorical_crossentropy',
  metrics=['accuracy']
)

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 256)               16384     
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 128)               32896     
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense_2 (Dense)             (None, 64)                8256      
                                                                 
 dense_3 (Dense)             (None, 5)                 325       
                                                                 
Total params: 57,861
Trainable params: 57,861
Non-traina

In [10]:
history = model.fit(
  X_train, y_train,
  validation_data = (X_test, y_test),
  epochs = 50,
  batch_size = 32
)

Epoch 1/50
63/63 [==============================] - 1s 6ms/step - loss: 1.4606 - accuracy: 0.3726 - val_loss: 1.1919 - val_accuracy: 0.6400
Epoch 2/50
63/63 [==============================] - 0s 3ms/step - loss: 0.9428 - accuracy: 0.6885 - val_loss: 0.5107 - val_accuracy: 0.8800
Epoch 3/50
63/63 [==============================] - 0s 3ms/step - loss: 0.3928 - accuracy: 0.8958 - val_loss: 0.1670 - val_accuracy: 0.9540
Epoch 4/50
63/63 [==============================] - 0s 3ms/step - loss: 0.1918 - accuracy: 0.9424 - val_loss: 0.0964 - val_accuracy: 0.9880
Epoch 5/50
63/63 [==============================] - 0s 3ms/step - loss: 0.1048 - accuracy: 0.9700 - val_loss: 0.0421 - val_accuracy: 0.9960
Epoch 6/50
63/63 [==============================] - 0s 3ms/step - loss: 0.0687 - accuracy: 0.9820 - val_loss: 0.0293 - val_accuracy: 0.9920
Epoch 7/50
63/63 [==============================] - 0s 3ms/step - loss: 0.0618 - accuracy: 0.9815 - val_loss: 0.0168 - val_accuracy: 1.0000
Epoch 8/50
63/63 [==

In [11]:
# save trained model
model.save("asl_landmark_anne.h5")
print("model saved!")

model saved!


In [12]:
# save label encoder
import pickle
with open("label_encoder.pkl", "wb") as f:
  pickle.dump(le, f)

print("label encoder saved!")

label encoder saved!


In [13]:
import cv2
import mediapipe as mp
import tensorflow as tf
import numpy as np
import sklearn
import pandas as pd

print("OpenCV:", cv2.__version__)
print("MediaPipe:", mp.__version__)
print("TensorFlow:", tf.__version__)
print("NumPy:", np.__version__)
print("Scikit-learn:", sklearn.__version__)
print("Pandas:", pd.__version__)

mpHands = mp.solutions.hands
print("Hands module loaded successfully ✅")


OpenCV: 4.8.1
MediaPipe: 0.10.8
TensorFlow: 2.12.0
NumPy: 1.23.5
Scikit-learn: 1.3.2
Pandas: 2.1.4
Hands module loaded successfully ✅


In [ ]:
# install below versions
# OpenCV: 4.8.1
# MediaPipe: 0.10.8
# TensorFlow: 2.12.0
# NumPy: 1.23.5
# Scikit-learn: 1.3.2
# Pandas: 2.1.4